In [23]:
import numpy as np

def scale_difference(i, j):
    """점수 차이를 선형적으로 1-9 스케일로 변환하는 함수."""
    if i == j:
        return 1  # 동일한 점수의 비교는 항상 1
    else:
        # 두 점수의 비율을 계산하고 1에서 9 사이로 스케일링
        min_value = min(i, j)
        max_value = max(i, j)
        ratio = max_value / min_value

        # 1과 2 사이의 비율을 1과 9 사이로 선형 스케일링
        # 1:1 비율은 1, 2:1 비율은 9로 매핑
        # 이 사이의 값은 선형 변환을 사용하여 계산
        scaled_value = 1 + (ratio - 1) / (2 - 1) * (5 - 1)
        return scaled_value if i >= j else 1 / scaled_value

# 주어진 중요도 점수
scores = np.array([2.86,3.91,2.95,3.46,3.65])

# 쌍대비교 행렬 생성
n = len(scores)
comparison_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        comparison_matrix[i, j] = scale_difference(scores[i], scores[j])


print(comparison_matrix )
# 가중치 계산 (일단 eigenvector 방식 사용)
eigenvalues, eigenvectors = np.linalg.eig(comparison_matrix)
max_eigenvalue_index = eigenvalues.argmax()
weights = np.abs(eigenvectors[:, max_eigenvalue_index]) / np.sum(np.abs(eigenvectors[:, max_eigenvalue_index]))

# 일관성 지수(CI)와 일관성 비율(CR) 계산
lambda_max = eigenvalues[max_eigenvalue_index].real
CI = (lambda_max - n) / (n - 1)

# 음수 CI 보정
if CI < 0:
    CI = 0

# Random Index (RI) 값
RI_dict = {1: 0, 2: 0, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45}
RI = RI_dict[n]

CR = CI / RI

print("가중치:", weights)
print(sum(weights))
print("CI:", CI)
print("CR:", CR)

# 결과 출력
if CR <= 0.1:
    print("일관성이 충분합니다.")
else:
    print("일관성이 부족합니다. 행렬을 재검토하세요.")


[[1.         0.40509915 0.88819876 0.54372624 0.47508306]
 [2.46853147 1.         2.30169492 1.52023121 1.28493151]
 [1.12587413 0.43446244 1.         0.59118236 0.51304348]
 [1.83916084 0.65779468 1.69152542 1.         0.81990521]
 [2.1048951  0.7782516  1.94915254 1.21965318 1.        ]]
가중치: [0.1163412  0.30287056 0.12773345 0.20752518 0.2455296 ]
1.0000000000000002
CI: 0.0005679953772965707
CR: 0.000507138729729081
일관성이 충분합니다.
